# SIMULATOR AND SUMMARY STATISTICS

In [ ]:
import os
#os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [12]:
# Load the minimum required library to run the functions
from Utils_functions import *
from numpy.random import uniform
from numpy import array, int64, where, concatenate, linspace, median, sum, log, zeros, mean, var, min, max, einsum
from scipy.integrate import cumulative_trapezoid
from scipy.signal import welch
import matplotlib.pyplot as plt
import torch
import numpy as np


In [2]:
dt = 1e-6# The time step size for the simulation. This is the amount of time that passes in the model for each step of the simulation
Sample_frequency = 25_000 # The sampling frequency. This is the number of samples that are recorded per unit time
DeltaT = 1/Sample_frequency # The sampling period. This is the amount of time that passes in the model between each sample that is recorded
TotalT = 1 # The total time for the simulation. This is the total amount of time that the simulation is intended to represent
n_sim = int(2e3)
t_corr = TotalT/50

In [3]:
time_steps_amount = int64(TotalT/dt) # Number of steps
sampled_point_amount = int64(TotalT/DeltaT) # Number of sampled points
sampling_delta_time_steps = int64(DeltaT/dt) # Number of steps between samples
t = linspace(0., TotalT,sampled_point_amount) # Time array

Chose the time characteristics of the input signal (total time, sample rate, numbero of simulations) and transform them into points info

In [4]:
# Limits for the parameters
prior_limits = [[1.5e4, 4e4], 
                [1e4, 140e4], 
                [3e-3, 16e-3], 
                [1.5e-2, 30e-2], 
                [1e-3, 6e-3], 
                [2e-2, 20e-2], 
                [0.5, 6], 
                [5.5, 15.5], 
                [1, 530]]

In [5]:
# Get the theta values from the prior in the different shapes
theta, theta_torch, prior_box = get_theta_from_prior(prior_limits, n_sim)

In [14]:
# Dummy call to force compilation
theta, theta_torch, prior_box = get_theta_from_prior(prior_limits, 1)
Simulator_noGPU(0.1,0.1,1,1,theta)

(array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32),
 array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32),
 array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32))

In [7]:
%%time
theta, theta_torch, prior_box = get_theta_from_prior(prior_limits, n_sim)
x_trace, f_trace, y_trace = Simulator_noGPU(dt, DeltaT, TotalT, n_sim, theta)

In [ ]:
# import _pickle as pickle
# with open('data/x_trace.pkl', 'wb') as f:
#     pickle.dump(x_trace, f)
    
# with open('data/f_trace.pkl', 'wb') as f:
#     pickle.dump(f_trace, f)

# with open('data/y_trace.pkl', 'wb') as f:
#     pickle.dump(y_trace, f)
    
# with open("data/theta.pkl", 'wb') as f:
#     pickle.dump(theta, f)

In [ ]:
S_mean, Fx, Fy, S_tot = compute_entropy_production(x_trace, y_trace, f_trace, theta, n_sim)

In [ ]:
for i in range(3):
    plt.plot(x_trace[i,:])

Simulating the x trajectory and the f trajectory to then obtain the summary statistics, in this case the autocorrellation and the cross correllation in combination with the reduced energy production.

I don't know which formula we can pick to compute the reduce energy production, from autocorrellation, from cross correllation, all together?

ADD: Write a function GET_SUMMARY_STATISTICS() which is a list of functions, each function for a summoner statistic

In [ ]:
%%time
list_stat = ["Cxx", "s_redx", "s_redf", "psdx"]
s = get_summary_statistics(list_stat, x_trace, f_trace, theta, DeltaT, 3, t, t_corr)
s.shape

Maybe we can select different summary statistics and usa some feature selection technique. If we use a lot of summary statistics maybe dimentionality reductions? PCA?
The idea is to use different summary statistics but to then to select the most important features of them deleting the correllations between them.

An example could be the Power spectral density for both x and f.
It can identify if there are any dominant frequencies in your data, or if the power is distributed across a range of frequencies, usefull for oscillatory behaviour

We can use directly the psds as input of the neural network or we can take some statistics about each one, like mean, median, max, min,variance, entropy, frequency of top power. We can think of more statistics

# INFERENCE STATISTICS

In [ ]:
from torch import Tensor, Size
from sbi import analysis as analysis
from sbi import utils as utils
from sbi.inference import SNPE, simulate_for_sbi, infer
from sbi.utils.user_input_checks import (
    check_sbi_inputs,
    process_prior,
    process_simulator,
)
from numpy import squeeze

In [ ]:
# Prior distribution for sbi
prior, num_parameters, prior_returns_numpy = process_prior(prior_box)

In [ ]:
# Call infer form sbi
infer = SNPE(prior=prior)
inferece = infer.append_simulations(theta_torch, s)

In [ ]:
# Train the model
density_estimator = infer.train()
posterior = infer.build_posterior(density_estimator)

In [ ]:
# Simulate the observation(s)
theta_true, theta_torch_true, prior_box_true = get_theta_from_prior(prior_limits, n_sim=1)
x_trace_true, f_trace_true, y_trace_true = Simulator_noGPU(dt, DeltaT, TotalT, 1, theta_true)

In [ ]:
# Sample from the posterior
s = get_summary_statistics(list_stat, x_trace_true, f_trace_true, theta, DeltaT, 3, t, t_corr)
samples = posterior.sample((int(1e5),), x=s)

In [ ]:
# Getting the some statistics from each marginal distribution
mean_params = np.array([])
mode_params = np.array([])

for i in range(9):
    # Retrive the samples for the parameter i
    params = samples[:,i].numpy()
    # Compute the mean
    mean_params = np.concatenate((mean_params, [np.mean(params)]))
    # Compute the mode
    hist, bin_edges = np.histogram(params, bins=int(np.sqrt(params.shape[0])))
    max_index = np.argmax(hist)
    mode = (bin_edges[max_index] + bin_edges[max_index+1])/2
    mode_params = np.concatenate((mode_params, [mode]))

mean_params = mean_params.reshape(9, 1, 1)
mode_params = mode_params.reshape(9, 1, 1)

In [ ]:
# Pairplot
label_theta = [r"$\mu_x$", r"$\mu_y$", r"$k_x$", r"$k_y$", r"$k_{int}$", r"$\tau$", r"$\varepsilon$", r"$D_x$", r"$D_y$"]
import matplotlib as mpl
_ = analysis.pairplot(samples, points=[theta_torch_true], 
                      limits=prior_limits, figsize=(10, 10), 
                      title = "Pairplot for the marginal posterior",
                      labels=label_theta)

In [ ]:
# Compute a new trace using the mode parameters
x_trace_est, f_trace_est, y_trace_est = Simulator_noGPU(dt, DeltaT, TotalT, 1, mode_params)

# ENTROPY PRODUCTION

These are the formulas used to compute the entropy production
$$
S = \sum_{t=0}^{N_{\text{timesteps}}-2} \left( \text{data}_{t+1} - \text{data}_{t} \right) \cdot F\left( \frac{\text{data}_{t+1} + \text{data}_{t}}{2} \right) \cdot D^{-1}\left( \frac{\text{data}_{t+1} + \text{data}_{t}}{2} \right)
$$
$$
\sigma = \frac{{\mu_y \cdot \varepsilon^2}}{{1 + k_y \cdot \mu_y \cdot \tau - \frac{{k_{int}^2 \cdot \mu_x \cdot \mu_y \cdot \tau^2}}{{1 + k_x \cdot \mu_x \cdot \tau}}}}
$$

In [ ]:
sigma_true, _, _, _ = compute_entropy_production(x_trace_true, y_trace_true, f_trace_true, theta_true, 1)
sigma_est, _, _, _ = compute_entropy_production(x_trace_est, y_trace_est, f_trace_est, mean_params, 1)

In [ ]:
print(sigma_true, sigma_est)
sigma_true/sigma_est

In [ ]:
sigma_true_2, _, = compute_entropy_2(theta_true, 1)
sigma_est_2_mean, _, = compute_entropy_2(mean_params, 1)
sigma_est_2_mode, _, = compute_entropy_2(mode_params, 1)
print("Estimates for the analytical entropy production:")
print(f"True: %.2f, Mean: %.2f, Mode: %.2f" % (sigma_true_2[0][0], sigma_est_2_mean[0][0], sigma_est_2_mode[0][0]))

In [ ]:
sigma_true_2, sigma_est_2_mode, sigma_est_2_mean

In [ ]:
# Get the statistics of sigma directly out of the samples
sigma_samples = array([])
#sanity_check = array([])

for i in range(samples.shape[0]):
    sample = samples[i,:].numpy().reshape(9,1,1)
    #sanity_check = concatenate((sanity_check, [sample[0][0][0]]))
    sigma = compute_entropy_2(list(sample), 1)[0][0][0]
    sigma_samples = concatenate((sigma_samples, [sigma]))

In [ ]:
plt.xlim(0, 1e4)
plt.vlines(sigma_true_2, 0, 1e3, color='r')
plt.xlabel(r"$\sigma$")
plt.ylabel("Frequency")
plt.title("Sigma from sampling")
plt.hist(sigma_samples, bins=int(np.sqrt(len(sigma_samples))));

# QUESTIONS
- per calcolare S_red come summary statistics (come fatto da Dresda nel file) usiamo i parametri, non è un bias poi usarla come summary statistics nel neural network durante la SBI?
- qual è la formula corretta per l'entropia? Cosa abbiamo usato noi?
- limiti per la prior distribution sui parametri